# Natural Language Processing

## 기본과제 3: Subword-level Language Model

> Reference 코드는 Solution 과 함께 공개됩니다.


### Introduction


* 본 과제의 목적은 서브워드 토큰화 (Subword Tokenization)의 필요성을 직접 느끼고 서브워드 토큰화 알고리즘 중 하나인 Byte Pair Encoding을 구현해봅니다.
* 서브워트 토큰화 기반 language model을 구현하면서 이전 과제의 Word-level language model과 비교해보는 시간을 갖겠습니다. 추가적으로 RNN을 LSTM으로 변경했을 때의 성능 차이에 대해 살펴보겠습니다.
* Subword-level language model을 구현하고, 주어진 데이터를 가공하여 모델을 학습한 후 학습된 언어 모델을 이용해 문장을 생성합니다.
* **ANSWER HERE** 이라고 작성된 부분을 채워 완성하시면 됩니다. 다른 부분의 코드를 변경하면 오류가 발생할 수 있습니다.

> 과제 완성 후 ipynb 파일을 제출해 주세요.<br>

### 0. 데이터 업로드


1. Boostcourse [기본 과제] Subword-level Language Model 에서 `wikitext-2.zip` 파일을 다운받습니다.
2. 본 Colab 환경에 `train.txt`, `dev.txt`, `test.txt` 파일을 업로드합니다.
3. `! ls` command 를 실행했을 때, `sample_data  test.txt  train.txt  valid.txt` 가 나오면 성공적으로 데이터 준비가 완료된 것 입니다.

In [155]:
! ls

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
'[Basic_2]_RNN_based_Language_Model 과제.ipynb'		      practice.ipynb
'[Basic_3]_Subword_level_Language_Model_ipynb의_사본.ipynb'   test.txt
 generate.txt						      train.txt
 model.pt						      valid.txt


In [156]:
path_train = '/opt/ml/RNN_homework/train.txt'
with open(path_train, 'r', encoding="utf8") as f:
    corpus_train = f.readlines()    

# train dataset 크기 확인
print(len(corpus_train))

# 처음 10 문장을 print 해 봅시다.
for sent in corpus_train[:10]:
    print(sent)

36718
 

 = Valkyria Chronicles III = 

 

 Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . <unk> the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " <unk> Raven " . 

 The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more <unk> for series newcom

### 1. 서브워드 토큰화의 필요성

💡 서브워드(Subword)는 무엇인가요?

서브워드는 하나의 단어를 여러개의 단위로 분리했을 때 하나의 단위를 나타냅니다. `subword`를 서브워드 단위로 나타낸 하나의 예시는 다음과 같습니다.

 * `sub` + `word`

`sub`라는 접두사와 `word`라고 하는 어근으로 나누어 `subword`라고 하는 단어를 2개의 서브 워드로 나타냈습니다.

이외에도 다양한 형태의 서브워드로 나타낼 수 있습니다. (e.g., `su` + `bword`, `s` + `ubword`, `subwor` + `d`)


💡 그럼 서브워드 토큰화(Subword tokenization)는 무엇인가요?

서브워드 토큰화는 말 그대로 서브워드 단위로 토큰화를 한다는 뜻입니다.
기본 과제 1에서 나온 단어단위 토큰화를 적용한 뒤, 서브워드 토큰화를 수행한 예시를 보겠습니다.

서브워드 토큰화를 적용했을 때는 다음과 같이 토큰화할 수 있습니다.

* Example 1
> "I have a meal" -> ['I', 'hav', 'e', 'a', 'me', 'al']
>
> "나는 밥을 먹는다" -> ['나', '는', '밥', '을', '먹는', '다']

단어단위가 아니라 그보다 더 잘게 쪼갠 서브워드 단위로 문장을 토큰화합니다.

위에서 말씀드린 것과 같이 여러가지 경우의 수가 가능합니다.

* Example 2
> "I have a meal" -> ['I', 'ha', 've', 'a', 'mea', 'l']
>
> "나는 밥을 먹는다" -> ['나', '는', '밥', '을', '먹', '는다']

그렇지만 기본적으로 공백을 넘어선 서브를 구성하진 않습니다.
예를 들어 다음과 같이 토큰화를 수행하진 않습니다.

* Example 3
> "I have a meal" -> ['Iha', 've', 'am', 'ea', 'l']
>
> "나는 밥을 먹는다" -> ['나는밥', '을먹', '는다']

(참고4: [Huggingface: subword-tokenization](https://huggingface.co/transformers/tokenizer_summary.html#subword-tokenization))

💡 Subword tokenization은 왜 필요한가요?

첫 번째 이유는 이 세상에 단어가 너무 많기 때문입니다.
이전 과제에서 사용했던 코드를 불러와 그 필요성을 생각해 봅시다.

In [157]:
import os
import torch

class Dictionary(object):
    def __init__(self):
        self.token2id = {}
        self.id2token = []

    def add_word(self, word):
        if word not in self.token2id:
            self.id2token.append(word)
            self.token2id[word] = len(self.id2token) - 1
        return self.token2id[word]

    def __len__(self):
        return len(self.id2token)

class Corpus(object):
    def __init__(self, path):
        self.dictionary = Dictionary()
        self.train = self.tokenize(os.path.join(path, 'train.txt'))
        self.valid = self.tokenize(os.path.join(path, 'valid.txt'))
        self.test = self.tokenize(os.path.join(path, 'test.txt'))

    def tokenize(self, path):
        """Tokenizes a text file."""
        assert os.path.exists(path)
        # Add words to the dictionary
        with open(path, 'r', encoding="utf-8") as f:
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    self.dictionary.add_word(word)

        # Tokenize file content
        with open(path, 'r', encoding="utf-8") as f:
            idss = []
            for line in f:
                words = line.split() + ['<eos>']
                ids = []
                for word in words:
                    ids.append(self.dictionary.token2id[word])
                idss.append(torch.tensor(ids).type(torch.int64))
            ids = torch.cat(idss)

        return ids

In [158]:
from typing import Union, Tuple
import torch
import torch.nn as nn
import torch.nn.functional as F

class RNNModel(nn.Module):
    def __init__(self, 
        rnn_type: str,
        vocab_size: int,
        embedding_size: int=200,
        hidden_size: int=200,
        num_hidden_layers: int=2,
        dropout: float=0.5
    ):
        super().__init__()
        self.rnn_type = rnn_type
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.num_hidden_layer = num_hidden_layers
        assert rnn_type in {'LSTM', 'GRU', 'RNN_TANH', 'RNN_RELU'}

        self.embedding = nn.Embedding(vocab_size, embedding_size)

        self.dropout = nn.Dropout(dropout)

        if rnn_type.startswith('RNN'):
            nonlinearity = rnn_type.split('_')[-1].lower()
            self.rnn = nn.RNN(
                embedding_size, 
                hidden_size, 
                num_hidden_layers,
                batch_first=True, 
                nonlinearity=nonlinearity,
                dropout=dropout
            )
        else:
            self.rnn = getattr(nn, rnn_type)(
                embedding_size,
                hidden_size,
                num_hidden_layers,
                batch_first=True,
                dropout=dropout
            )

        self.projection = nn.Linear(hidden_size, vocab_size)

    def forward(
        self, 
        input: torch.Tensor,
        prev_hidden: Union[torch.Tensor, Tuple[torch.Tensor, torch.Tensor]]
    ):
        """ RNN 모델의 forward 함수 구현
        위의 그림과 __init__ 함수 내 주석을 참고하여 forward 함수를 구현하세요.

        Hint 2: RNN 모델에선 Dropout을 곳곳에 적용하는 것이 성능이 좋다고 알려져 있습니다.
                예를 들어, Embedding 이후와 Projection 전에도 적용할 수 있습니다.
        Hint 2: 최종 확률값을 구하기 위해서 Projection 이후에 F.log_softmax를 사용하면 됩니다.

        Arguments:
        input -- 토큰화 및 배치화된 문장들의 텐서
                    dtype: torch.long
                    shape: [batch_size, sequence_lentgh]
        prev_hidden -- 이전의 hidden state
                    dtype: torch.float
                    shape: RNN, GRU - [num_layers, batch_size, hidden_size]
                           LSTM - ([num_layers, batch_size, hidden_size], [num_layers, batch_size, hidden_size])

        Return:
        log_prob -- 다음 토큰을 예측한 확률에 log를 취한 값
                    dtype: torch.float
                    shape: [batch_size, sequence_length, vocab_size]
        next_hidden -- 이후의 hidden state
                    dtype: torch.float
                    shape: RNN, GRU - [num_layers, batch_size, hidden_size]
                           LSTM - ([num_layers, batch_size, hidden_size], [num_layers, batch_size, hidden_size])
        """
        ### YOUR CODE HERE
        ### ANSWER HERE ###
        emb = self.dropout(self.embedding(input))
        output, next_hidden = self.rnn(emb, prev_hidden)
        log_prob = self.projection(self.dropout(output)).log_softmax(dim=-1)

        ### END YOUR CODE
        
        assert list(log_prob.shape) == list(input.shape) + [self.vocab_size]
        assert prev_hidden.shape == next_hidden if self.rnn_type != 'LSTM' \
          else prev_hidden[0].shape == next_hidden[0].shape == next_hidden[1].shape
        
        return log_prob, next_hidden
    
    def init_hidden(self, batch_size: int):
        """ 첫 hidden state를 반환하는 함수 """
        
        weight = self.projection.weight
        
        if self.rnn_type == 'LSTM':
            return (weight.new_zeros(self.num_hidden_layer, batch_size, self.hidden_size),
                    weight.new_zeros(self.num_hidden_layer, batch_size, self.hidden_size))
        else:
            return weight.new_zeros(self.num_hidden_layer, batch_size, self.hidden_size)
    
    @property
    def device(self):   # 현재 모델의 device를 반환하는 프로퍼티
        return self.projection.weight.device

말뭉치의 문장들을 단어단위 토큰화를 해보고 단어들의 개수를 세어보겠습니다

In [159]:
corpus = Corpus('/opt/ml/RNN_homework/')
vocab_size = len(corpus.dictionary)
print(vocab_size)

33278


이전 과제에 사용된 임베딩의 크기는 200이므로 단어 임베딩에 사용된 매개변수의 수는 33278 x 200 (6,655,600개)입니다.
그렇다면, RNN 모델에 사용되는 weight의 parameter 개수는 몇개인지 간단한 함수를 이용해 확인해보겠습니다

In [160]:
model = RNNModel('RNN_TANH', vocab_size)

In [161]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Word embedding parameter 개수: {count_parameters(model.embedding)}")
print(f"RNN parameter 개수: {count_parameters(model.rnn)}")

Word embedding parameter 개수: 6655600
RNN parameter 개수: 160800


💡 RNN 층의 매개변수 개수와, 임베딩 매개변수 개수를 비교해보면 임베딩 매개변수의 개수가 RNN층의 매개변수 수보다 압도적으로 많습니다.

단어단위 임베딩을 사용하는 경우 학습에 사용되는 말뭉치의 크기가 커질수록 등장하는 단어가 더더욱 많아져 임베딩의 매개변수는 더 커지게 되고 전체 매개변수 대비 단어 임베딩이 차지하는 비중은 매우 높아집니다.

✨ 이런 매개변수 비중의 비대칭성을 해결하기 위해 처음에는 문자단위 토큰화(character-level tokenization) 방법이 주목을 받았습니다. 
말 그대로 하나의 글자를 기준으로 토큰화을 하는건데요.
이전 예시를 문자단위 토큰화를 하면 다음과 같습니다.

"I have a meal" -> ['I', 'h', 'a', 'v', 'e', 'a', 'm', 'e', 'a', 'l']
"나는 밥을 먹는다" -> ['나', '는', '밥', '을', '먹', '는', '다']

그러나, 문자단위 토큰화 역시 지나치게 긴 Sequence 길이, 성능 저하 등의 문제를 겪으며 서브워드 토큰화가 각광을 받게 되었습니다.




💡서브워드 토큰화가 가지는 두번째 장점은 Out-of-Vocabulary (OoV) 문제가 없다는 점입니다.

학습 데이터에서 등장하지 않은 단어는 모두 Unknown 토큰 [UNK]로 처리됩니다. 이는 테스트 과정 중에 처음 보는 단어를 모두 [UNK]로 모델의 입력을 넣게 되면서 전체적으로 모델의 성능이 저하될 수 있습니다.

그러나 서브워드 단위로 자르게 된다면 최악의 경우에도 문자단위로 토큰화가 진행됩니다. 이는 서브워드 토큰화는 현재 가지고 있는 Vocab으로 해당 단어가 토큰화할 수 없다면 그 단어를 서브워드 단위로 쪼개 평가하기 때문입니다.

따라서 서브워드 토큰화기는 가장 작은 문자 단위로 서브워드 토큰화가 가능하기 때문에 OoV 문제가 발생하지 않습니다. 

### 2. Byte Pair Encoding (BPE)
> 이 섹터에서는 파이썬 표준 라이브러리 (Python Standard Library)만 사용하세요.

대표적인 서브워드 토큰화 방법인 Byte pair encoding을 구현해봅시다. BPE의 정확한 알고리즘은 [논문](https://arxiv.org/pdf/1508.07909.pdf)의 3페이지 algorithm 1에 제시되어 있습니다. 각 문항과 주석의 지시사항을 확인하고 BPE를 구현해보세요.


### 2-A) BPE Vocab 만들기

BPE의 Vocab을 만드는 것은 간단합니다. 단순히 가장 많이 등장하는 연속한 짝을 찾아 추가하는 것 입니다.
다음과 같은 말뭉치가 있다고 가정해 봅시다.

```
low lower lowest newest
```

우선은 공백을 제외한 모든 문자를 Vocab에 추가하고 각 단어의 끝에 WORD_END "`_`" 붙여 단어를 구분지어 봅시다.

```
Vocab: d e i l n o r s t w _
[ l o w _ ], [ l o w e r _ ], [ l o w e s t _ ], [ w i d e s t _ ]  
```

이때 가장 많이 등장한 연속한 두 토큰을 찾아 Vocab에 추가하고 두 토큰을 붙입니다. 이 경우에는 `l o`가 세번 등장하여 가장 많았으니 `lo`로 붙여 Vocab에 추가합니다.

```
Vocab: d e i l n o r s t w _ lo
[ lo w _ ], [ lo w e r _ ], [ lo w e s t _ ], [ w i d e s t _ ] 
```

다음은 `lo w`가 세번 등장하므로 `low`를 추가합니다.

```
Vocab: d e i l n o r s t w _ lo low
[ low _ ], [ low e r _ ], [ low e s t _ ], [ w i d e s t _ ] 
```

다음은 `e s`가 두번 등장하므로 `es`를 추가합니다.

```
Vocab: d e i l n o r s t w _ lo low es
[ low _ ], [ low e r _ ], [ low es t _ ], [ w i d es t _ ] 
```

다음은 `es t`가 두번 등장하므로 `est`를 추가합니다.

```
Vocab: d e i l n o r s t w _ lo low es est
[ low _ ], [ low e r _ ], [ low est _ ], [ w i d est _ ] 
```

다음은 `est _`가 두번 등장하므로 `est_`를 추가합니다.

```
Vocab: d e i l n o r s t w _ lo low es est est_
[ low _ ], [ low e r _ ], [ low est_ ], [ w i d est_ ] 
```

`est_`는 est로 단어가 끝난다는 것을 알려주는 서브워드가 됩니다. 일반적으로 est가 나오면 단어가 끝나니 합리적입니다.

이러한 과정을 통해서 모든 단어가 추가되거나 원하는 Vocab 크기에 도달할 때까지 서브워드를 통합하여 추가하는 과정을 반복하면 됩니다. 알고리즘을 참고하여 `build_bpe`를 작성해 봅시다.

In [162]:
from typing import List
import collections, re
from collections import Counter




# 단어 끝을 나타내는 문자
WORD_END = '_'

def build_bpe(
    corpus: List[str],
    max_vocab_size: int
) -> List[int]:
    """ BPE Vocab 만들기
    Byte Pair Encoding을 통한 Vocab 생성을 구현하세요.
    단어의 끝은 '_'를 사용해 주세요.
    이때 id2token을 서브워드가 긴 길이 순으로 정렬해 주세요.
    
    Note: 만약 모든 단어에 대해 BPE 알고리즘을 돌리게 되면 매우 비효율적입니다.
          왜냐하면 대부분의 단어는 중복되기 때문에 중복되는 단어에 대해서는 한번만 연산할 수 있다면 매우 효율적이기 때문입니다.
          따라서 collections 라이브러리의 Counter를 활용해 각 단어의 빈도를 구하고,
          각 단어에 빈도를 가중치로 활용하여 BPE를 돌리면 시간을 획기적으로 줄일 수 있습니다.
          물론 이는 Optional한 요소입니다.

    Arguments:
    corpus -- Vocab을 만들기 위한 단어 리스트
    max_vocab_size -- 최대 vocab 크기

    Return:
    id2token -- 서브워드 Vocab. 문자열 리스트 형태로 id로 token을 찾는 매핑으로도 활용 가능
    """
    ### YOUR CODE HERE
    def get_stats(vocab):
        pairs=collections.defaultdict(int)
        for word,freq in vocab.items():
            word=word.split(' ')
            for i in range(len(word)-1):
                pairs[word[i]+word[i+1]] +=freq

        return pairs

    def merge_vocab(vocab,pair):
        v_out={}
        for word,freq in vocab.items():
            word_list=word.split(' ')
            for i in range(len(word_list)-1):
                if pair == word_list[i]+word_list[i+1]:
                    sub=word_list[i]+' '+word_list[i+1]
                    word= word.replace(sub,pair)
                    break
            
            if word not in v_out:
                v_out[word]=freq
            else:
                v_out[word]+= freq

        return v_out
    
    corpus= Counter(corpus)
    vocab={}
    
    for word, freq in corpus.items():
        s=''
        for letter in word:
            s+=letter
            s+=' '
        s+=('_')
        vocab[s] = freq
        
      
    id2token=[]
    id2token.append('_')
    for word in set(corpus):
        for letter in word:
            if letter not in id2token and letter != ' ':
                id2token.append(letter)    
    while len(id2token) < max_vocab_size:
        pairs= get_stats(vocab)
        if len(pairs) ==0 :
            break
        if len(pairs) ==1 :
            best=list(pairs.keys())[0]
        else:
            
            best=max(pairs, key= pairs.get)        
        vocab=merge_vocab(vocab,best)
        if best not in id2token:
            id2token.append(best)

    

    ### ANSWER HERE ###

    ### END YOUR CODE
    id2token = sorted(id2token,key=len, reverse=True)
    
    
    
    return id2token

**2-A 문제에 대한 테스트 코드**

In [163]:
print ("======Building BPE Vocab Test Case======")

# 첫번째 테스트
corpus = ['abcde']
vocab = build_bpe(corpus, max_vocab_size=15)
assert sorted(vocab, key=len, reverse=True) == vocab, \
       "id2token을 서브워드 길이가 긴 순으로 정렬해 주세요."
print("첫번째 테스트 통과!")

# 두번째 테스트
corpus = ['low'] * 5 + ['lower'] * 2 + ['newest'] * 6 + ['widest'] * 3
vocab = set(build_bpe(corpus, max_vocab_size=19))
assert vocab > {'est_', 'low', 'newest_', \
              'i', 'e', 'n', 't', 'd', 's', 'o', 'l', 'r', 'w', WORD_END} and \
       "low_" not in vocab and "wi" not in vocab and "id" not in vocab, \
       "BPE 결과가 기대한 결과와 다릅니다."
print("두번째 테스트 통과!")

# 세번째 테스트
corpus = ['aaaaaaaaaaaa', 'abababab']
vocab = set(build_bpe(corpus, max_vocab_size=8))
assert vocab == {'aaaaaaaa', 'aaaa', 'abab', 'aa', 'ab', 'a', 'b', WORD_END}, \
       "BPE 결과가 기대한 결과와 다릅니다."
print("세번째 테스트 통과!")

# 네번째 테스트
corpus = ['abc', 'bcd']
vocab = build_bpe(corpus, max_vocab_size=10000)
assert len(vocab) == 10, \
       "BPE 결과가 기대한 결과와 다릅니다."
print("네번째 테스트 통과!")

print("모든 테스트 통과!")

======Building BPE Vocab Test Case======
첫번째 테스트 통과!
두번째 테스트 통과!
세번째 테스트 통과!
네번째 테스트 통과!
모든 테스트 통과!


### 2-B) BPE 인코딩
만들어진 Vocab으로 텍스트 인코딩하는 방법은 몇 가지가 있습니다. 가장 쉬운 방법은 앞에서부터 토큰화하되 가장 긴 것부터 욕심쟁이 기법(Greedy Search)으로 먼저 매칭하는 방법입니다.

```
Vocab: bcde ab cd bc de a b c d e _
abcde ==> ab cd e _
```

이 방법은 최적의 인코딩을 보장하진 않지만 긴 단어를 빠르게 인코딩하는 것이 가능합니다.

두번째 방법은 가장 길게 매칭되는 것을 전체 텍스트에 대해 먼저 토큰화하는 방법입니다.

```
Vocab: bcde ab cd bc de a b c d e _
abcde ==> a bcde _
```

두번째 방법은 첫번째 방법보다 느리지만 텍스트를 좀 더 짧게 인코딩하는 것이 가능합니다.

이 과제에서는 두번째 방법을 이용하여 BPE 인코딩을 구현해봅시다.

In [164]:
def encode(
    sentence: str,
    id2token: List[str]
) -> List[int]:
    """ BPE 인코더
    문장을 받아 BPE 토큰화를 통하여 고유 id의 수열로 바꿉니다.
    문장은 공백으로 단어단위 토큰화되어있다고 가정하며, Vocab은 sentence의 모든 문자를 포함한다고 가정합니다.
    찾을 수 있는 가장 긴 토큰부터 바꿉니다.
    
    Note: WORD_END를 빼먹지 마세요.

    Arguments:
    sentence -- 인코드하고자 하는 문장
    id2token -- build_bpe를 통해 만들어진 Vocab
    
    Return:
    token_ids -- 인코드된 토큰 id 수열
    """
    token_ids=[]
    sentences=sentence.split(' ')
    for sentence in sentences:
        sentence+='_'
        for token in vocab:
            if token in sentence:
                sentence= sentence.replace(token,str(vocab.index(token)))
        token_ids+=sentence
        
        

    ### YOUR CODE HERE
    ### ANSWER HERE ###

    ### END YOUR CODE
    for i in range(len(token_ids)):
        token_ids[i] = int(token_ids[i])
    return token_ids

**2-B 문제에 대한 테스트 코드**

In [165]:
print ("======Encoding Test Case======")

# 첫번째 테스트
vocab = ['bcc', 'bb', 'bc', 'a', 'b', 'c', WORD_END]
assert encode('abbccc', vocab) == [3, 4, 0, 5, 6], \
       "BPE 인코딩 결과가 기대한 결과와 다릅니다."
print("첫번째 테스트 통과!")

# Second test
vocab = ['aaaa', 'aa', 'a', WORD_END]
assert len(encode('aaaaaaaa aaaaaaa', vocab)) == 7, \
       "BPE 인코딩 결과가 기대한 결과와 다릅니다."
print("두번째 테스트 통과!")

print("모든 테스트 통과!")


======Encoding Test Case======
첫번째 테스트 통과!
두번째 테스트 통과!
모든 테스트 통과!


### 2-C) BPE 디코딩

BPE로 인코딩된 것을 디코딩하는 것은 간단합니다.
그저 해당 id를 해당하는 서브워드로 만든 뒤 합치면됩니다.
WORD_END는 공백으로 처리하면 쉽습니다.

```
[ 196 62 20 6 ] ==> [ I_ li ke_ it_ ] ==> "I_like_it_" ==> "I like it " ==> "I like it"  
```

In [166]:
def decode(
    token_ids: List[int],
    id2token: List[str]
) -> str:
    """ BPE 디코더
    BPE로 토큰화된 id 수열을 받아 문장으로 바꿉니다.
    단어단위 토큰화에서의 문장 복원은 단순히 공백을 사이에 넣는 디코딩을 사용합니다.
    문장 끝의 공백은 잘라냅니다.
    
    Arguments:
    token_ids -- 디코드하고자하는 토큰 id 수열
    id2token -- build_bpe를 통해 만들어진 Vocab

    Return:
    sentence  -- 디코드된 문장
    """

    ### YOUR CODE HERE
    ### ANSWER HERE ###
    sentence=''
    for ids in token_ids:
        sentence+=id2token[ids]
    
    sentences=sentence.replace('_',' ')
    
    sentences = sentences[:-1]
    ### END YOUR CODE
    return sentences

**2-C 문제에 대한 테스트 코드**

In [167]:
print ("======Decoding Test Case======")
# First test
vocab = ['bcc', 'bb', 'bc', 'a', 'b', 'c', WORD_END]
assert decode([3, 4, 0, 5, 6], vocab) == 'abbccc', \
           "BPE 디코딩 결과가 기대한 결과와 다릅니다."
print("첫번째 테스트 통과!")

# Second test
vocab = ['aaaa', 'aa', 'a', WORD_END]
assert decode([0, 0, 3, 0, 1, 2, 3], vocab) == 'aaaaaaaa aaaaaaa', \
           "BPE 디코딩 결과가 기대한 결과와 다릅니다."
print("두번째 테스트 통과!")

======Decoding Test Case======
첫번째 테스트 통과!
두번째 테스트 통과!


### 3. Transformers 라이브러리를 활용한 서브워드 토큰화

✨ 위에서 작성한 BPE 구현체를 통해 서브워드 토큰화의 원리를 알 수 있지만, 위의 구현체를 실제로 사용하기에는 난점이 존재합니다.
왜냐하면 BPE Vocab을 만드는 과정은 매우 오랜 시간이 걸리기 때문입니다.
다양한 토큰화기(tokenizer)를 직접 구현하고 학습하는 것은 매우 비용이 크기 때문에, 라이브러리를 활용하여 토큰화기를 사용하는 방법을 알아봅시다.

[Transformer](https://huggingface.co/docs/transformers/index) 라이브러리는 다양한 Transformer 구현체를 총망라한 라이브러리입니다.
Transfomer 외에도 다양한 토큰화기를 지원하는데, 이미 학습된 서브워드 토큰화기 역시 쉽게 불러올 수 있습니다.

(참고5: [Huggingface: subword tokenization](https://huggingface.co/transformers/tokenizer_summary.html#subword-tokenization))

In [168]:
! pip install transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [170]:
from transformers import BertTokenizerFast

# BERT 모델에서 사용하는 토큰화를 가져옵니다.
# https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained(
    "bert-base-cased",
    unk_token='<unk>',
    eos_token='<eos>'
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


**Question**

Transformers에서 제공되는 BertTokenizerFast는 모든 조합을 만들 수 있는 서브워드 기반 토큰화기임에도 불구하고 위와 같이 Unknown 토큰을 받을 수 있다. 서브워드 토큰화기에서 Unknown 토큰이 발생할 수 있는 상황은 무엇이 있을까?

**Answer**

토큰이 vocabulary에 존재하지 않는경우 unk 토큰을 받는다

In [171]:
# 서브워드 토큰화 예시
print(tokenizer.tokenize('Boostcamp AI Tech'))
token_ids = tokenizer("Boostcamp AI Tech", add_special_tokens=False).input_ids
print(token_ids)
print(tokenizer.decode(token_ids))

print(tokenizer.tokenize('qwerklhfa asdfkwej \n adsfesdff'))
token_ids = tokenizer("qwerklhfa asdfkwej", add_special_tokens=False).input_ids
print(token_ids)
print(tokenizer.decode(token_ids))

['Bo', '##ost', '##cam', '##p', 'AI', 'Tech']
[9326, 15540, 24282, 1643, 19016, 7882]
Boostcamp AI Tech
['q', '##wer', '##k', '##l', '##h', '##fa', 'as', '##d', '##f', '##k', '##we', '##j', 'ads', '##fe', '##s', '##d', '##ff']
[186, 12097, 1377, 1233, 1324, 8057, 1112, 1181, 2087, 1377, 7921, 3361]
qwerklhfa asdfkwej


이 토큰화기는 `##`을 통하여 현 단어가 이전 단어와 연결되어 있는지를 알려주고 있습니다. 이 토큰화기를 기반으로 다시 모델을 선언하고 parameter의 개수를 살펴보겠습니다.

In [172]:
vocab_size = len(tokenizer)
print(vocab_size)
subword_model = RNNModel('RNN_TANH', vocab_size)

28998


In [173]:
print(f"임베딩 매개변수 개수: {count_parameters(subword_model.embedding)}")
print(f"RNN층 매개변수 개수: {count_parameters(subword_model.rnn)}")

임베딩 매개변수 개수: 5799600
RNN층 매개변수 개수: 160800


이전에 비해 임베딩 매개변수 개수는 확연히 줄어들었습니다.

6,655,600개 -> 5,799,600개

그에 비하여 이 토큰화기는 이전 토큰화기와 달리 학습 데이터에 없었던 영어 단어가 새로 나오더라도 토큰화가 가능합니다. 

그러면 이제 서브워드 토큰화 기반의 언어 모델 성능을 살펴봅시다.

### 4. 서브워드 기반 Language Model 학습
앞서 확인해본 `transformers` 라이브러리 기반 토큰화기를 활용하여 서브워드 기반 Lanuage Model을 학습시켜 봅시다. 기본 과제 2를 참고하여 구현해봅시다. 



In [174]:
### YOUR CODE HERE
### 기본 과제 2를 참고해서 Language model 학습 코드를 작성해 보세요.
### ANSWER HERE ###


import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np


from tqdm.notebook import tqdm
from torch.nn.utils import clip_grad_norm_
from typing import Union,Tuple
from transformers import BertTokenizerFast

def bptt_batchify(data,batch_size,sequence_length):
    
    drop_data_len = len(data)%(batch_size*sequence_length)
    data=data[:len(data)-drop_data_len]
    data=torch.tensor(data)
    data= data.view((batch_size,-1,sequence_length))
    batches=np.transpose(data,(1,0,2))
    
    print('batchifiy done')
    return batches
    
    
    
    
def train(model : RNNModel, data : torch.Tensor, lr:float):
    model.train()
    
    batch_size = data.shape[1]
    total_loss =0.
    hidden = model.init_hidden(batch_size)
    progress_bar = tqdm(data,desc="Train")
    for bid, batch in enumerate(progress_bar, start=1):
        batch=batch.to(model.device)
        output,hidden = model(batch,hidden)
        if model.rnn_type == 'LSTM':
            hidden = tuple(tensor.detach() for tensor in hidden)
        else:
            hidden = hidden.detach()
        loss=F.nll_loss(output[:, :-1, :].transpose(1, 2), batch[:, 1:])
        model.zero_grad()
        loss.backward()
        clip_grad_norm_(model.parameters(),0.25)
        for param in model.parameters():
            param.data.add_(param.grad,alpha=-lr)
        
        total_loss += loss.item()
        current_loss = total_loss /bid
        progress_bar.set_description(f"Train - loss {current_loss:5.2f} | ppl {math.exp(current_loss):8.2f} | lr {lr:02.2f}", refresh=False)
        
        
        
    

@torch.no_grad()
def evaluate(model:RNNModel, data:torch.Tensor):
    model.eval()
    batch_size=data.shape[1]
    total_loss=0
    hidden=model.init_hidden(batch_size)
    progress_bar = tqdm(data,desc='Eval')
    for bid,batch in enumerate(progress_bar, start=1):
        batch=batch.to(model.device)
        output,hidden=model(batch,hidden)
        if model.rnn_type == "LSTM":
            hidden = tuple(tensor.detach() for tensor in hidden)
        else:
            hidden = hidden.detach()
        
        loss = F.nll_loss(output[:, :-1, :].transpose(1, 2), batch[:, 1:])
        total_loss += loss.item()
        current_loss = total_loss /bid
        progress_bar.set_description(f"Eval - loss {current_loss:5.2f} | ppl {math.exp(current_loss):8.2f}", refresh=False)
    
    return loss
        



with open('/opt/ml/RNN_homework/train.txt', 'r', encoding="utf8") as f:
    corpus_train = f.readlines()   
with open('/opt/ml/RNN_homework/valid.txt', 'r', encoding="utf8") as f:
    corpus_val = f.readlines()   
with open('/opt/ml/RNN_homework/test.txt', 'r', encoding="utf8") as f:
    corpus_test = f.readlines()   
    
# BERT tokenizer ready
tokenizer = BertTokenizerFast.from_pretrained(
    "bert-base-cased",
    unk_token='<unk>',
    eos_token='<eos>'
)

#token to id
train_token_ids = sum(tokenizer(corpus_train, add_special_tokens=False).input_ids,[])
val_token_ids = sum(tokenizer(corpus_val, add_special_tokens=False).input_ids,[])
test_token_ids =  sum(tokenizer(corpus_test, add_special_tokens=False).input_ids,[])
#print(tokenizer.decode(token_ids))


#datasets ready
train_data = bptt_batchify(train_token_ids,batch_size=16,sequence_length=64)
val_data = bptt_batchify(val_token_ids,batch_size=16,sequence_length=64)
test_data = bptt_batchify(test_token_ids,batch_size=16,sequence_length=64)


#model ready###################################
vocab_size = len(tokenizer)
model = RNNModel('LSTM', vocab_size)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=model.to(device)


#train start
lr=20
num_epoch=30
best_loss=None



for epoch in range(1,num_epoch+1):
    train(model,train_data,lr)
    val_loss = evaluate(model,val_data)
    print('-' * 89)
    print(f'| End of epoch {epoch:2d} | valid loss {val_loss:5.2f} | valid ppl {math.exp(val_loss):8.2f}')
    print('-' * 89)
    
    if not best_loss or val_loss < best_loss:
        torch.save(model,'model.pt')
        best_loss = val_loss
    else:
        lr /=4.0

print('Train done!!!!')
    



### END YOUR CODE

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Token indices sequence length is longer than the specified maximum sequence length for this model (655 > 512). Running this sequence through the model will result in indexing errors


batchifiy done
batchifiy done
batchifiy done



-----------------------------------------------------------------------------------------
| End of epoch  1 | valid loss  5.82 | valid ppl   336.81
-----------------------------------------------------------------------------------------



-----------------------------------------------------------------------------------------
| End of epoch  2 | valid loss  5.63 | valid ppl   278.17
-----------------------------------------------------------------------------------------



-----------------------------------------------------------------------------------------
| End of epoch  3 | valid loss  5.54 | valid ppl   254.64
-----------------------------------------------------------------------------------------


...


-----------------------------------------------------------------------------------------
| End of epoch 26 | valid loss  4.98 | valid ppl   145.20
-----------------------------------------------------------------------------------------



-----------------------------------------------------------------------------------------
| End of epoch 27 | valid loss  4.98 | valid ppl   144.89
-----------------------------------------------------------------------------------------



-----------------------------------------------------------------------------------------
| End of epoch 28 | valid loss  4.98 | valid ppl   144.96
-----------------------------------------------------------------------------------------



-----------------------------------------------------------------------------------------
| End of epoch 29 | valid loss  4.97 | valid ppl   143.74
-----------------------------------------------------------------------------------------



-----------------------------------------------------------------------------------------
| End of epoch 30 | valid loss  4.97 | valid ppl   143.55
-----------------------------------------------------------------------------------------
Train done!!!!


### 5. 학습한 언어 모델로 문장 생성

앞서 학습한 모델을 불러와서 문장을 생성해봅시다.
기본 과제 2와 똑같이 generate.txt를 저장하면 됩니다.


In [175]:
model =torch.load('model.pt', map_location=device)
model.rnn.flatten_parameters()
test_loss = evaluate(model, test_data)
print('=' * 89)
print(f'| End of training | test loss {test_loss:5.2f} | test ppl {math.exp(test_loss):8.2f}')
print('=' * 89)


| End of training | test loss  5.13 | test ppl   168.97


In [176]:
### YOUR CODE HERE
### 기본 과제 2를 참고해서 Langauge model로 문장을 생성하는 코드를 작성해 보세요.
### ANSWER HERE ###
from tqdm.notebook import trange

num_words = 1000
temperature = 1.0

hidden = model.init_hidden(1)
input = torch.randint(vocab_size, (1, 1), dtype=torch.long).to(device)
outputs = []

for i in trange(num_words, desc="Generation"):
    with torch.no_grad():
        log_prob, hidden = model(input, hidden)

    weights = (log_prob.squeeze() / temperature).exp()
    token_id = torch.multinomial(weights, 1)
    outputs.append(token_id.item())
    input = token_id.unsqueeze(0)
    
outputs =tokenizer.decode(outputs)

with open('generate.txt', 'w') as fd:
    fd.write(' '.join(outputs).replace('<eos>', '\n'))

### END YOUR CODE